In this notebook, I will explore the Titanic dataset as well as review a couple of statistics techniques in R.

# Introduction

 1. [Very Quick Summary](##very-quick-summary)
 2. [Survival Relationships](##survival-relationships)
 3. [Missing Values](##missing-values)
 4. [Feature Engineering](##feature-engineering)
 5. [Predicting](##predicting)

## Very Quick Summary

We will start our analysis by loading a couple of useful libraries and get a quick grasp of the data we've just loaded.

In [ ]:
library(ggplot2)
library(dplyr,warn.conflicts = FALSE)
library(formattable,warn.conflicts = FALSE)

tr = read.csv("../input/train.csv")
pr = read.csv("../input/test.csv")
pr$Survived <- NA
mergedData <- rbind(tr,pr)

formattable(summary(tr))

Display the top lines. Gives us another perspective. It will probably be hard to get something out of:

 - Cabin: it look like there are a lot of null values
 - Tickets: the string pattern/meaning will be hard to transform into value

In [ ]:
formattable(head(tr,100))

## Survival Relationships

Check the relationship between every variables and survivability.

### Pclass

There are very few class categories, but the survivability ratio between the three of them is very clear.

In [ ]:
trPclass <- tr %>% group_by(Pclass) %>% summarize(cnt=n(),surv=mean(Survived)) %>% arrange(-surv) %>% as.data.frame()
formattable(trPclass)

### Sex

Note that there are almost two males for one female. Female has a much better chance of survival.

In [ ]:
trSex <- tr %>% group_by(Sex) %>% summarize(cnt=n(),surv=mean(Survived)) %>% arrange(-surv) %>% as.data.frame()
formattable(trSex)



### Age

We have 177 age null values, we will need to take care of at a later stage. Let's just look into the distribution here.

The age is probably not very predictive as there are no clear relationship between the two variables. We will know more about predictability when we run the model.

In [ ]:
ggplot(subset(tr,!is.na(Age)), aes(x = Age, fill = factor(Survived))) + geom_histogram(bins=15,position = 'dodge')



### Siblings

 - Having siblings do improve your chances of survival

In [ ]:
#library(ggvis)

#on_hover <- function(x) {
#  if(is.null(x)) return(NULL)
#  mgi_symbol <- x
#  mgi_symbol
#}
#ggvis(tr,props(x = ~SibSp, y = ~Survived, y2 = 0)) +  mark_rect(props(width := 10))

ggplot(tr, aes(x = SibSp, fill = factor(Survived))) + geom_bar(position='dodge')

### Parents/Children

Small families do have a better chance of survival.

In [ ]:
ggplot(tr, aes(x = Parch, fill = factor(Survived))) + geom_bar(position='dodge')

### Fare

We can note a slight increase of survivability when the ticket fare increases. However as most of the tickets have been bought at a low price, this will just help us a little.

In [ ]:
ggplot(tr, aes(x = Fare, fill = factor(Survived))) + geom_histogram(bins=15,position='dodge')

### Embarked

Only three significant categories and 20% of our population can be classified with a higher chance of survival.
We have two missing values.

In [ ]:
trEmbarked <- tr %>% group_by(Embarked) %>% summarize(cnt=n(),surv=mean(Survived)) %>% arrange(-surv) %>% as.data.frame()
formattable(trEmbarked)

## Feature Enginnering

We've just looked into the raw data. Let's see if we can enrich it with the other string columns.

###Name

Name is a string variable. String variables can be rich, but they should be transformed to categorical or -even better- numerical variables to be useful.

From our first lines, we can deduce everyone's title is between a comma and a dot. Let's extract this first and see.

In [ ]:
mergedData$title <- gsub('(.*, )|(\\..*)', '', mergedData$Name)
mergedData$title[mergedData$title == "Ms"] <- "Miss"
mergedData$title[mergedData$title == "Mlle"] <- "Miss"
mergedData$title[!mergedData$title %in% c("Miss","Mrs","Mr")] <- "Other"

trTitle <- mergedData[!is.na(mergedData$Survived),] %>% group_by(title) %>% summarize(cnt=n(),surv=mean(Survived)) %>% arrange(-surv) %>% as.data.frame()
formattable(trTitle)

### Cabin

As mentioned earlier, Cabin is probably a column hard to get any value from due to the null values. Let's just count.
Well maybe, we were a bit quick in our judgement. The numbers of non-null values are small, but it seems that it could have some predictive potential.

In [ ]:
mergedData$cabinCharCnt <- sapply(mergedData$Cabin,function(x) nchar(as.character(x))> 0)
trCabin <- mergedData[!is.na(mergedData$Survived),] %>% group_by(cabinCharCnt) %>% summarize(cnt=n(),surv=mean(Survived)) %>% arrange(-cnt) %>% as.data.frame()
formattable(trCabin)


### Tickets

Tickets are quite hard to figure out. We will just make an attempt to remove the numbers and see what we get.
With 660 without any value and nothing clear out of the other tiny categories, we will just give up on this field.

In [ ]:
mergedData$ticketsCnt <- sapply(mergedData$Ticket,function(x) nchar(as.character(x)))
#Count number of character in tickets
#trTickets <- tr %>% group_by(ticketsCnt) %>% summarize(cnt=n(),surv=mean(Survived)) %>% arrange(-cnt) %>% as.data.frame()
    
#Remove digits
mergedData$ticketLetter <- gsub('[0-9/\\.]+', '', toupper(mergedData$Ticket))
trTickets <- mergedData[!is.na(mergedData$Survived),] %>% group_by(ticketLetter) %>% summarize(cnt=n(),surv=mean(Survived)) %>% arrange(-cnt) %>% as.data.frame()
formattable(trTickets)
    

## Missing Values

### Embarked

In [ ]:
formattable(subset(mergedData,nchar(as.character(Embarked)) == 0))

In [ ]:
library(rpart)

mergedData$Name <- NULL
mergedData$Ticket <- NULL
mergedData$Cabin <- NULL
mergedData$ticketsCnt <- NULL
mergedData$ticketLetter <- NULL
trClean1 <- mergedData
class_emb_mod <- rpart(Embarked ~ . - Survived, data=subset(trClean1,nchar(as.character(Embarked)) > 0), method="class", na.action=na.omit) 
emb_pred <- predict(class_emb_mod, subset(trClean1,nchar(as.character(Embarked)) == 0))
trClean1$Embarked[nchar(as.character(trClean1$Embarked)) == 0] <- colnames(emb_pred)[apply(emb_pred, 1, which.max)]


### Age

In [ ]:
formattable(head(subset(tr,is.na(Age))))

In [ ]:
library(mice)

trClean2 <- trClean1
# perform mice imputation, based on random forests
miceMod <- mice(trClean2[,!names(trClean2) %in% "Survived"], method="rf",printFlag=FALSE) 
# generate the completed data
trClean2[,!names(trClean2) %in% "Survived"] <- complete(miceMod)
#anyNA(miceOutput)
head(trClean2)

## Predicting

### Random Forest

In [ ]:
library(randomForest)
set.seed(415)
trClean <- trClean2
#lapply(trClean, class)
trClean$Survived <- as.factor(trClean$Survived)
trClean$title <- as.factor(trClean$title)

trRF <- trClean[!is.na(trClean$Survived),]
fit <- randomForest(Survived ~ . - PassengerId,
                      data=trRF, 
                      importance=TRUE, 
                      ntree=2000)
fit$confusion
varImpPlot(fit)

In [ ]:
prRF <- trClean[is.na(trClean$Survived),]
prediction <- predict(fit, prRF)
prRF$Survived <-prediction
formattable(prRF)